<a href="https://colab.research.google.com/github/ChessEngineUS/hybrid-quantum-protein-folding/blob/main/notebooks/advanced_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔬 Advanced Structure Analysis

This notebook demonstrates advanced analysis techniques for protein structures.

**What you'll learn:**
- 📊 Energy landscape visualization
- 🧬 Contact map analysis
- 📐 Ramachandran plot generation
- 📈 Convergence analysis
- 🎯 Structure quality metrics

**Prerequisites:** Complete the quickstart notebook first.

In [ ]:
# Installation
import os
if os.path.exists('hybrid-quantum-protein-folding'):
    !rm -rf hybrid-quantum-protein-folding
!git clone -q https://github.com/ChessEngineUS/hybrid-quantum-protein-folding.git
%cd hybrid-quantum-protein-folding
!pip install -q -r requirements-colab.txt
!pip install -q -e .
print('✅ Setup complete!')

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import pdist, squareform
from sklearn.decomposition import PCA

os.chdir('/content/hybrid-quantum-protein-folding')
from hqpf.models import HybridModel
from hqpf.data.benchmarks import load_benchmark_protein, AA_TO_IDX

sns.set_style('whitegrid')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'✅ Imports complete | Device: {device}')

## 1. Generate Multiple Structures

Generate an ensemble of structures for analysis.

In [ ]:
# Load protein
protein = load_benchmark_protein('helix_12', 'test')
sequence = protein.to_tensor(AA_TO_IDX, device=device)

# Initialize model
model = HybridModel(
    n_residues=len(sequence),
    n_qubits=12,
    use_quantum=False,
    device=device
)

# Generate ensemble
print('🔬 Generating structure ensemble...')
model.eval()
with torch.no_grad():
    outputs = model(sequence, n_candidates=20)

structures = [s.cpu().numpy() for s in outputs['all_structures']]
energies = [e.cpu().item() for e in outputs['all_energies']]
best_idx = np.argmin(energies)

print(f'✅ Generated {len(structures)} structures')
print(f'Energy range: {min(energies):.3f} to {max(energies):.3f}')
print(f'Best structure: #{best_idx+1} with energy {energies[best_idx]:.3f}')

## 2. Energy Landscape Analysis

Visualize energy distribution across the ensemble.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Energy histogram
axes[0].hist(energies, bins=15, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(energies[best_idx], color='red', linestyle='--', linewidth=2, label='Best')
axes[0].set_xlabel('Energy', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[0].set_title('Energy Distribution', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Energy vs rank
sorted_energies = sorted(enumerate(energies), key=lambda x: x[1])
ranks = [i for i, _ in sorted_energies]
sorted_vals = [e for _, e in sorted_energies]
axes[1].plot(range(1, len(sorted_vals)+1), sorted_vals, 'o-', linewidth=2, markersize=8)
axes[1].set_xlabel('Rank', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Energy', fontsize=12, fontweight='bold')
axes[1].set_title('Energy Landscape', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print('✅ Energy landscape plotted')

## 3. Contact Map Analysis

Compute and visualize residue-residue contacts.

In [ ]:
def compute_contact_map(structure, threshold=8.0):
    """Compute binary contact map."""
    n = len(structure)
    contacts = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            dist = np.linalg.norm(structure[i] - structure[j])
            if dist < threshold:
                contacts[i, j] = 1
                contacts[j, i] = 1
    return contacts

# Compute for best structure
best_structure = structures[best_idx]
contacts = compute_contact_map(best_structure, threshold=8.0)

# Visualize
plt.figure(figsize=(10, 8))
plt.imshow(contacts, cmap='YlOrRd', interpolation='nearest')
plt.colorbar(label='Contact')
plt.xlabel('Residue Index', fontsize=12, fontweight='bold')
plt.ylabel('Residue Index', fontsize=12, fontweight='bold')
plt.title(f'Contact Map (threshold=8.0 Å)\n{protein.sequence}', fontsize=14, fontweight='bold')

# Add sequence labels
ticks = np.arange(len(protein.sequence))
plt.xticks(ticks, list(protein.sequence), fontsize=10)
plt.yticks(ticks, list(protein.sequence), fontsize=10)

plt.tight_layout()
plt.show()

n_contacts = int(np.sum(contacts) / 2)
print(f'✅ Found {n_contacts} contacts within 8.0 Å')

## 4. Structural Diversity Analysis

Analyze diversity in the structural ensemble.

In [ ]:
# Compute pairwise RMSD matrix
def compute_rmsd(s1, s2):
    """Compute RMSD between two structures."""
    return np.sqrt(np.mean((s1 - s2)**2))

n_struct = len(structures)
rmsd_matrix = np.zeros((n_struct, n_struct))

for i in range(n_struct):
    for j in range(i+1, n_struct):
        rmsd = compute_rmsd(structures[i], structures[j])
        rmsd_matrix[i, j] = rmsd
        rmsd_matrix[j, i] = rmsd

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# RMSD heatmap
im = axes[0].imshow(rmsd_matrix, cmap='viridis')
plt.colorbar(im, ax=axes[0], label='RMSD (Å)')
axes[0].set_xlabel('Structure Index', fontsize=12)
axes[0].set_ylabel('Structure Index', fontsize=12)
axes[0].set_title('Pairwise RMSD Matrix', fontsize=14, fontweight='bold')

# RMSD distribution
upper_triangle = rmsd_matrix[np.triu_indices_from(rmsd_matrix, k=1)]
axes[1].hist(upper_triangle, bins=20, color='teal', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('RMSD (Å)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('RMSD Distribution', fontsize=14, fontweight='bold')
axes[1].axvline(np.mean(upper_triangle), color='red', linestyle='--', label=f'Mean: {np.mean(upper_triangle):.2f} Å')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f'✅ Structural diversity: Mean RMSD = {np.mean(upper_triangle):.3f} ± {np.std(upper_triangle):.3f} Å')

## 5. Principal Component Analysis

Reduce dimensionality and visualize conformational space.

In [ ]:
# Flatten structures for PCA
flattened = np.array([s.flatten() for s in structures])

# Perform PCA
pca = PCA(n_components=2)
coords_2d = pca.fit_transform(flattened)

# Visualize
plt.figure(figsize=(12, 8))
scatter = plt.scatter(coords_2d[:, 0], coords_2d[:, 1], 
                     c=energies, cmap='coolwarm', s=200, 
                     edgecolors='black', linewidths=2, alpha=0.8)

# Highlight best structure
plt.scatter(coords_2d[best_idx, 0], coords_2d[best_idx, 1], 
           c='gold', s=500, marker='*', edgecolors='black', linewidths=2, 
           label='Best Structure', zorder=5)

plt.colorbar(scatter, label='Energy')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=12, fontweight='bold')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', fontsize=12, fontweight='bold')
plt.title('Conformational Space (PCA)', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f'✅ PCA: {pca.explained_variance_ratio_[0]*100:.1f}% + {pca.explained_variance_ratio_[1]*100:.1f}% = {sum(pca.explained_variance_ratio_)*100:.1f}% variance explained')

## 📊 Summary

You've completed advanced structure analysis! Key findings:

- Energy landscape shows structure quality distribution
- Contact maps reveal spatial interactions
- RMSD analysis quantifies structural diversity
- PCA reduces conformational space to interpretable dimensions

**Next:** Try these analyses on your own proteins!